In [ ]:
import pandas_datareader as pdr
import numpy as np 
from numpy import array 
from sklearn.preprocessing import MinMaxScaler
import pandas as pd 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import math 
from sklearn.metrics import mean_squared_error 
import matplotlib.pyplot as plt

In [ ]:
key=
df=pdr.get_data_tiingo('AAPL',api_key=key)


In [ ]:
df.to_csv("AAPL.csv")

In [ ]:
df=pd.read_csv("AAPL.csv")

In [ ]:
df2=df.reset_index()['close']

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df2)

In [ ]:

scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df2).reshape(-1,1))

In [ ]:
df1.shape

In [ ]:
#splitting dataset
training=int(len(df1)*0.65)
test_size=len(df1)-training
train_data,test_data=df1[0:training],df1[training:len(df1)]

In [ ]:
def create_dataset(dataset,time_step=1):
    dataX,dataY=[],[]
    for i in range(len(dataset)-time_step-1):
        a=dataset[i:(i+time_step),0]
        dataX.append(a)
        dataY.append(dataset[i+time_step,0])
    return np.array(dataX),np.array(dataY)


In [ ]:
time_step=100
X_train,y_train=create_dataset(train_data,time_step)
X_test,y_test=create_dataset(test_data,time_step)

In [ ]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
model =Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64,verbose=1)

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
 math.sqrt(mean_squared_error(y_train,train_predict))

In [ ]:
 math.sqrt(mean_squared_error(y_test,test_predict))

In [ ]:
look_back=100
trainPredictPlot=np.empty_like(df1)
trainPredictPlot[:,:]=np.nan
trainPredictPlot[look_back:len(train_predict)+look_back,:]=train_predict


testPredictPlot=np.empty_like(df1)
testPredictPlot[:,:]=np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1,:]=test_predict

plt.plot(scaler.inverse_transform(df1))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show

In [ ]:
x_input=test_data[340:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input= list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
lst_output=[]
n_step=100
i=0
while(i<30):
    if (len(temp_input)>100):
        x_input=np.array(temp_input[1:])
        print("{}day input{}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input=x_input.reshape((1,n_step,1))
        yhat=model.predict(x_input,verbose=0)
        print("{}day output{}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        lst_output.extend(yhat.tolist())
        i+=1
    else:
        x_input=x_input.reshape(1,n_step,1)
        yhat=model.predict(x_input,verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        i+=1
        
print(lst_output)

In [ ]:
day_new=np.arange(1,101)

In [ ]:
day_pred=np.arange(101,131)

In [ ]:
len(df1)

In [ ]:
df3=df1.tolist()
df3.extend(lst_output)
plt.plot(df3[1000:])

In [ ]:
# plt.plot(day_new,scaler.inverse_transform(df1[1157:]))
# plt.plot(day_pred,scaler.inverse_transform(lst_output))